In [1]:
from typing import List, Optional
import json
import os
import sys

if "linux" in sys.platform:
    os.environ["LC_AL"] = "C.UTF-8"
    os.environ["LANG"] = "C.UTF-8"
else:
    os.environ["LC_AL"] = "en_US.utf-8"
    os.environ["LANG"] = "en_US.utf-8"

import numpy as np
import tweepy
from aitextgen import aitextgen

ModuleNotFoundError: No module named 'tweepy'

In [ ]:
def new_auth():
    consumer_key = "..."
    consumer_secret = "..."
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    redirect_url = auth.get_authorization_url()
    print(redirect_url)

In [ ]:
def get_access_token():
    oauth_token = "..."
    oauth_verifier = "..."
    auth.request_token = { 'oauth_token' : oauth_token,
                           'oauth_token_secret' : oauth_verifier }
    print(auth.get_access_token(oauth_verifier))

In [ ]:
def authenticate(consumer_key=None, consumer_secret=None, access_key=None, access_secret=None):
    consumer_key = consumer_key or os.environ["CONSUMER_KEY"]
    consumer_secret = consumer_secret or os.environ["CONSUMER_SECRET"]
    access_key = access_key or os.environ["ACCESS_KEY"]
    access_secret = access_secret or os.environ["ACCESS_SECRET"]

    # Authenticate with the Twitter API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    return api

In [ ]:
import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.DEBUG
    )
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
ai = aitextgen(model="pytorch_model.bin",
               config="config.json",
               vocab_file="aitextgen-vocab.json",
               merges_file="aitextgen-merges.json",
               to_gpu=False)

In [ ]:
ai.generate(n=50, prompt="@quemerogoku")

In [ ]:
import re
def new_tweet_text(temperature=0.7):
    # generate a tweet
    generated_text = ai.generate_one(temperature=temperature)
    while len(generated_text) >= 280:
        generated_text = ai.generate_one(temperature=temperature)
    
    if 'https://t.co' in generated_text:
        generated_text = re.sub('https\:\/\/t\.co.*', '', generated_text)
    
    print(generated_text)
    return generated_text

In [ ]:
api = authenticate()

In [ ]:
api.update_status(new_tweet_text())

In [ ]:
from random import randint
from time import sleep


class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if status.author.id == 190461974:
            seconds = randint(60, 300)
            print(f"new tweet. will wait {seconds} seconds and then tweet")
            sleep(seconds)
            api.update_status(new_tweet_text())

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth=api.auth, listener=myStreamListener)
myStream.filter(follow=["190461974"])
#myStream.filter(track=["python"])